In [11]:
pip install pygame

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 26.0
[notice] To update, run: C:\Users\denne\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [12]:
import pygame
import sys

# ------------------ SETUP ------------------
pygame.init()
WIDTH, HEIGHT = 900, 500
WIN = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Unfair Platformer – One Level")

CLOCK = pygame.time.Clock()
FPS = 60
GRAVITY = 1

WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
RED = (220, 50, 50)
BLUE = (80, 80, 255)
GREEN = (50, 200, 50)

# ------------------ PLAYER ------------------
player = pygame.Rect(50, 380, 30, 40)
vel_x, vel_y = 0, 0
on_ground = False

# ------------------ PLATFORMS ------------------
platforms = [
    pygame.Rect(0, 420, 900, 80),
    pygame.Rect(150, 350, 120, 20),
    pygame.Rect(350, 300, 120, 20),
    pygame.Rect(550, 260, 120, 20),
]

# Fake falling ground
fake_platforms = [
    {"rect": pygame.Rect(250, 380, 120, 20), "falling": False, "vy": 0},
    {"rect": pygame.Rect(650, 330, 120, 20), "falling": False, "vy": 0},
]

# ------------------ TRAPS ------------------
spikes = [
    pygame.Rect(450, 400, 40, 20),
    pygame.Rect(490, 400, 40, 20),
]

# Hidden spike zones (invisible)
hidden_spikes = [
    pygame.Rect(180, 410, 80, 10),
    pygame.Rect(600, 410, 80, 10),
]

# ------------------ GOAL ------------------
goal = pygame.Rect(830, 200, 40, 40)

# ------------------ RESET ------------------
def reset():
    global vel_x, vel_y
    player.x, player.y = 50, 380
    vel_x = vel_y = 0
    for fp in fake_platforms:
        fp["falling"] = False
        fp["vy"] = 0
        fp["rect"].y = fp["rect"].y  # reset position implicitly

# ------------------ MAIN LOOP ------------------
def main():
    global vel_x, vel_y, on_ground

    reset()

    while True:
        CLOCK.tick(FPS)
        WIN.fill(WHITE)

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()

        # ------------------ INPUT ------------------
        keys = pygame.key.get_pressed()
        vel_x = 0
        if keys[pygame.K_a]:
            vel_x = -5
        if keys[pygame.K_d]:
            vel_x = 5
        if keys[pygame.K_SPACE] and on_ground:
            vel_y = -15

        # ------------------ PHYSICS ------------------
        vel_y += GRAVITY
        player.x += vel_x
        player.y += vel_y
        on_ground = False

        # ------------------ PLATFORM COLLISION ------------------
        for p in platforms:
            if player.colliderect(p) and vel_y > 0:
                player.bottom = p.top
                vel_y = 0
                on_ground = True

        # Fake platform logic
        for fp in fake_platforms:
            if player.colliderect(fp["rect"]) and vel_y > 0 and not fp["falling"]:
                fp["falling"] = True
            if not fp["falling"] and player.colliderect(fp["rect"]) and vel_y > 0:
                player.bottom = fp["rect"].top
                vel_y = 0
                on_ground = True

        # Fake platform falling
        for fp in fake_platforms:
            if fp["falling"]:
                fp["vy"] += GRAVITY
                fp["rect"].y += fp["vy"]

        # ------------------ DEATH ------------------
        for s in spikes:
            if player.colliderect(s):
                reset()

        for hs in hidden_spikes:
            if player.colliderect(hs):
                reset()

        if player.y > HEIGHT:
            reset()

        # Invisible ceiling kill (unfair)
        if 500 < player.x < 650 and player.y < 200:
            reset()

        # ------------------ WIN CONDITION ------------------
        if player.colliderect(goal):
            print("YOU WIN (somehow).")
            pygame.quit()
            sys.exit()

        # ------------------ DRAW ------------------
        pygame.draw.rect(WIN, BLUE, player)

        for p in platforms:
            pygame.draw.rect(WIN, BLACK, p)

        for fp in fake_platforms:
            pygame.draw.rect(WIN, BLACK, fp["rect"])

        for s in spikes:
            pygame.draw.rect(WIN, RED, s)

        pygame.draw.rect(WIN, GREEN, goal)

        pygame.display.update()

main()   


YOU WIN (somehow).


SystemExit: 